In [1]:
from aad import B1530Lib

#help(B1530Lib)

In [2]:
B1530Lib.print_devices()
b = B1530Lib.B1530()

ASRL3::INSTR
ASRL4::INSTR
GPIB0::INTFC
GPIB0::18::INSTR


In [3]:
# 2.5V Pulse of a length of 0.1s with 0.1s between each, measure its value
b.chan[2].waveform = B1530Lib.Pulse(voltage=2.5, interval=100e-3, edges=1e-3, length=100e-3)
b.chan[2].measurement = b.chan[2].waveform.measure(mode='voltage', range='5V', average_time=1e-3, sample_rate=10e-3)

# For fun, decreasing ramp
b.chan[3].waveform = B1530Lib.Waveform(pattern=[[0, 2.5], [b.chan[2].waveform.get_total_duration(), 0]])

# Dont start the measurement before the input voltage is set
b.chan[1].measurement = b.chan[2].waveform.measure(mode='current', range='10mA', average_time=1e-3, sample_rate=10e-3, when_established=True)

# Run twice for fun
b.configure(repeat=2)

In [4]:
b.exec()

In [5]:
print("  T (s) ", '\t', "  U (V)")
for i in range(len(b.chan[2].measurement.result)):
  for time, meas in b.chan[2].measurement.get_result(i):
    print("{:.6f}".format(time), '\t', "{:.6f}".format(meas))
  print("*" * 26)

print('=' * 26)
print("  T (s) ", '\t', "  I (A)")
for i in range(len(b.chan[1].measurement.result)):
  for time, meas in b.chan[1].measurement.get_result(i):
    print("{:.6f}".format(time), '\t', "{:.6f}".format(meas))
  print("*" * 26)
  

  T (s)  	   U (V)
0.000500 	 -0.000711
0.010500 	 -0.000802
0.020500 	 -0.000706
0.030500 	 -0.000676
0.040500 	 -0.000734
0.050500 	 1.249834
0.060500 	 2.500016
0.070500 	 2.500296
0.080500 	 2.500204
0.090500 	 2.500184
0.100500 	 2.500290
0.110500 	 2.500185
0.120500 	 2.500278
0.130500 	 2.500340
0.140500 	 2.500161
0.150500 	 2.500290
0.160500 	 -0.000689
0.170500 	 -0.000668
0.180500 	 -0.000588
0.190500 	 -0.000688
**************************
0.202500 	 -0.000628
0.212500 	 -0.000617
0.222500 	 -0.000567
0.232500 	 -0.000569
0.242500 	 -0.000647
0.252500 	 1.249797
0.262500 	 2.500277
0.272500 	 2.500243
0.282500 	 2.500580
0.292500 	 2.500350
0.302500 	 2.500337
0.312500 	 2.500217
0.322500 	 2.500389
0.332500 	 2.500512
0.342500 	 2.500492
0.352500 	 2.500340
0.362500 	 -0.000676
0.372500 	 -0.000684
0.382500 	 -0.000729
0.392500 	 -0.000586
**************************
--------------------------
  T (s)  	   I (A)
0.051500 	 -0.003078
0.061500 	 -0.003078
0.071500 	 -0.003078


A resistor of 820 Ω is measured for a voltage of 2.5V : 2.5/820 = 30.48mA, OK  
 
There are 0s because the next measurement starts after 'start_delay's of the end of the last *measurements*, and not the last *pulse*...